<a href="https://colab.research.google.com/github/DCODE-ARMY/GAN/blob/main/Data_Efficient_Gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Jul 13 09:22:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#use centralStorageStrategy
%matplotlib inline
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from google.colab.patches import cv2_imshow
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import time
from tqdm import tqdm
import tensorflow_addons as tfa #used for spectral normalaization
from sklearn.utils import shuffle
import PIL
import PIL.Image


# TF_GPU_ALLOCATOR=cuda_malloc_async

In [ ]:
#total image 1228/32= 38 || 1228/16=76 || 1228/26=46
batch_size = 32
num_channels = 3
num_classes = 5
kid_image_size = 75
image_size = 256
noise_size = 256
channel_axis=3
steps_per_epoc=38
epochh=1
divide_array=np.array([6])

policy = 'color,translation,cutout'# color,

# adaptive discriminator augmentation
max_translation = 0.125
max_rotation = 0.125
max_zoom = 0.25
# target_accuracy = 0.85
# integration_steps = 1000

In [ ]:

def gene(data_dir='/content/drive/MyDrive/scrapped data',batch_size=batch_size,resize=(image_size,image_size)):
  cat=['laptop', 'keyboard', 'mouse', 'headphone', 'monitor']

  img_path_list=[]
  labels=[]

  #getting path to all images
  for i in cat:
    path=os.path.join(data_dir,i)
    for j in os.listdir(path):
      img_path_list.append(os.path.join(path,j))

  img_path_list=shuffle(img_path_list)

  for i in img_path_list:
    # print(cat.index(i.split('/')[-2]))
    labels.append(cat.index(i.split('/')[-2]))


#converting labels to onehot encoded vectors  
  # labels=tf.keras.utils.to_categorical(labels,5)
 

  img_list=[]
  labels_list=[]
 
  for i,j in enumerate(img_path_list):
    # img=cv2.resize(cv2.imread(j,cv2.IMREAD_UNCHANGED),resize,interpolation=cv2.INTER_AREA)
    img=np.array(PIL.Image.open(j))
    # tensor = tf.image.resize(tensor, (128,128))
    # tensor = tf.subtract(tf.divide(tensor, 127.5), 1)

    # img=img.astype(np.float32)
   
    img=tf.keras.layers.Resizing(image_size,image_size,interpolation='gaussian',crop_to_aspect_ratio=False)(img)
    # img=img.astype(np.float32)
    # img=(img-127.5)/127.5
    img = tf.subtract(tf.divide(img, 127.5), 1)
    # assert not np.any(np.isnan(img))
    img_list.append(img)
    
    labels_list.append(labels[i])

    if len(img_list)==batch_size:
      
      data=np.array(img_list,dtype=np.float32),np.expand_dims(np.array(labels_list,dtype=np.int8),axis=1)#not necessarily convert to tensor
      yield data
      img_list.clear()
      labels_list.clear()



#Tensorspec can be used to declare numpy signature but with dtype specified in tf

dataset=tf.data.Dataset.from_generator(gene,output_signature=(tf.TensorSpec(shape=(batch_size, image_size, image_size, channel_axis),dtype=tf.dtypes.float32),tf.TensorSpec(shape=(batch_size,1),dtype=tf.dtypes.int8)))
dataset=dataset.prefetch(buffer_size=tf.data.AUTOTUNE).repeat(1000)

In [ ]:
def val_gene(data_dir='/content/drive/MyDrive/scrapped data/val',batch_size=batch_size,resize=(image_size,image_size)):
  cat=['laptop', 'keyboard', 'mouse', 'headphone', 'monitor']

  img_path_list=[]
  labels=[]

  #getting path to all images
  for i in cat:
    path=os.path.join(data_dir,i)
    for j in os.listdir(path):
      img_path_list.append(os.path.join(path,j))

  img_path_list=shuffle(img_path_list)

  for i in img_path_list:
    # print(cat.index(i.split('/')[-2]))
    labels.append(cat.index(i.split('/')[-2]))


#converting labels to onehot encoded vectors  
  # labels=tf.keras.utils.to_categorical(labels,5)
 

  img_list=[]
  labels_list=[]
 
  for i,j in enumerate(img_path_list):
    # img=cv2.resize(cv2.imread(j,cv2.IMREAD_UNCHANGED),resize,interpolation=cv2.INTER_AREA)
    img=np.array(PIL.Image.open(j))
    # tensor = tf.image.resize(tensor, (128,128))
    # tensor = tf.subtract(tf.divide(tensor, 127.5), 1)

    # img=img.astype(np.float32)
   
    img=tf.keras.layers.Resizing(image_size,image_size,interpolation='gaussian',crop_to_aspect_ratio=False)(img)
    # img=img.astype(np.float32)
    # img=(img-127.5)/127.5
    img = tf.subtract(tf.divide(img, 127.5), 1)
    # assert not np.any(np.isnan(img))
    img_list.append(img)
    
    labels_list.append(labels[i])

    if len(img_list)==batch_size:
      
      data=np.array(img_list,dtype=np.float32),np.expand_dims(np.array(labels_list,dtype=np.int8),axis=1)#not necessarily convert to tensor
      yield data
      img_list.clear()
      labels_list.clear()



#Tensorspec can be used to declare numpy signature but with dtype specified in tf

val_dataset=tf.data.Dataset.from_generator(gene,output_signature=(tf.TensorSpec(shape=(batch_size, image_size, image_size, channel_axis),dtype=tf.dtypes.float32),tf.TensorSpec(shape=(batch_size,1),dtype=tf.dtypes.int8)))
val_dataset=val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE).repeat(1000)

In [ ]:
def conv2d_bn(x,filter,kernel_size,stride,padding='same',activation=tf.keras.layers.LeakyReLU(alpha=0.2),is_initializer=True,is_regularizer=False,initializer= tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05,)
,regularizer=None,name=None):
  if name is not None:
    bn_name = name + '_bn'
    conv_name = name + '_conv'
  else:
    bn_name = None
    conv_name = None

  


  if is_initializer and is_regularizer:
    
    x=tfa.layers.SpectralNormalization(layers.Conv2D(filters=filter,kernel_size=kernel_size,strides=stride,padding=padding,activation=activation,kernel_initializer=initializer,regularizer=regularizer,name=conv_name))(x)

  elif is_regularizer:
    
    x=tfa.layers.SpectralNormalization(layers.Conv2D(filters=filter,kernel_size=kernel_size,strides=stride,padding=padding,activation=activation,regularizer=regularizer,name=conv_name))(x)

  elif is_initializer:
    
    x=tfa.layers.SpectralNormalization(layers.Conv2D(filters=filter,kernel_size=kernel_size,strides=stride,padding=padding,activation=activation,kernel_initializer=initializer,name=conv_name))(x)

  else:
    x=tfa.layers.SpectralNormalization(layers.Conv2D(filters=filter,kernel_size=kernel_size,strides=stride,padding=padding,activation=activation,name=conv_name))(x)
   
  

  
  return x


In [ ]:
def discriminator():
  # initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=.05)

  # Create the discriminator.
  inp =layers.Input(shape=[image_size, image_size,channel_axis])

  label_inp_layer=layers.Input(shape=(1,))
  label_inp=layers.Embedding(num_classes,16)(label_inp_layer)

  label_inp=tfa.layers.SpectralNormalization(layers.Dense(16,activation=tf.keras.layers.LeakyReLU(alpha=0.2) )) (label_inp)

  label_inp=layers.Flatten()(label_inp)
 
  label_inp=layers.RepeatVector(4096)(label_inp) #try using 16 embedding
  
  
  # label_inp= tf.repeat(label_inp.numpy(), repeats=[image_size * image_size])

  
  label_inp=layers.Reshape((image_size,image_size,1))(label_inp)

  
  x = layers.concatenate([inp, label_inp],axis=-1)

  #filter,kernel_size,stride,padding,activation,initializer,regularizer,dropout_val

  # x=conv2d_bn(x,64,3,1,padding='same')

  # batch_size,128,128,32
  x=conv2d_bn(x,64,3,2,padding='same')


  x=conv2d_bn(x,32,1,1,padding='valid')

  x=conv2d_bn(x,192,3,1,padding='valid')
  x=layers.Dropout(0.2)(x)

  x=conv2d_bn(x,80,1,1,padding='valid')

  # batch_size,128,128,64
  x=conv2d_bn(x,64,3,1,padding='valid') #maxpool_2

  x=conv2d_bn(x,32,1,1,padding='valid')

  x=conv2d_bn(x,64,5,1,padding='same')
  # x=layers.Dropout(0.4)(x)

  x=conv2d_bn(x,80,1,1,padding='valid')

  x=conv2d_bn(x,64,3,2,padding='same')

  



  
  
  #block 1
  #batch_size,35 x 35 x 256
 
  branch1x1=conv2d_bn(x,64,1,1)

  branch5x5=conv2d_bn(x, 48, 1, 1)
  branch5x5 = conv2d_bn(branch5x5, 32, 5, 1)
  
  branch3x3dbl = conv2d_bn(x, 64, 1, 1)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 43, 5, 1)
  # branch3x3dbl=layers.Dropout(0.4)(branch3x3dbl)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 1)
  
  branch_pool = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
  branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
  x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed0')

  x=conv2d_bn(x,128,1,1)
  x=conv2d_bn(x,196,3,2)

  x=layers.Dropout(0.2)(x)
  #block 2
  #batch_size,35 x 35 x 256
  branch1x1=conv2d_bn(x,64,1,1)

  branch5x5=conv2d_bn(x, 48, 1, 1)
  branch5x5 = conv2d_bn(branch5x5, 64, 5, 1)

  branch3x3dbl = conv2d_bn(x, 64, 1, 1)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 1)
  # branch3x3dbl=layers.Dropout(0.2)(branch3x3dbl)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 5, 1)
  # branch3x3dbl=layers.Dropout(0.2)(branch3x3dbl)
 
  branch_pool = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
  branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
  x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed1')

  x=layers.Dropout(0.2)(x)

  # x=conv2d_bn(x, 64, 5, 2)

  #block 3
  #batch_size,35 x 35 x 256
  branch1x1=conv2d_bn(x,64,1,1)

  branch5x5=conv2d_bn(x, 48, 1, 1)
  branch5x5 = conv2d_bn(branch5x5, 32, 5, 1)

  branch3x3dbl = conv2d_bn(x, 64, 1, 1)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 1)
  # branch3x3dbl=layers.Dropout(0.4)(branch3x3dbl)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 1)

  branch_pool = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
  branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
  x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed2')


  
  x=layers.Dropout(0.2)(x)
  #block 3
  #batch_size,17 x 17 x 768
  branch3x3 = conv2d_bn(x, 196, 3, 2,padding='valid')

  branch3x3dbl = conv2d_bn(x, 64, 1, 1)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 1,)
  # branch3x3dbl=layers.Dropout(0.4)(branch3x3dbl)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 2,padding='valid')
  
  branch_pool = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

  x = layers.concatenate([branch3x3, branch3x3dbl,branch_pool], axis=channel_axis, name='mixed3')
  # x=layers.Dropout(0.4)(x)

  # mixed 4: 17 x 17 x 768
  branch1x1 = conv2d_bn(x, 192, 1, 1)

  branch7x7 = conv2d_bn(x, 128, 1, 1)
  branch7x7 = conv2d_bn(branch7x7, 128, (1,7), 1)
  # branch7x7=layers.Dropout(0.2)(branch7x7)
  branch7x7 = conv2d_bn(branch7x7, 128, (7,1), 1)

  branch7x7dbl = conv2d_bn(x, 128, 1, 1)
  branch7x7dbl = conv2d_bn(branch7x7dbl, 128, (7,1), 1)
  # branch7x7dbl=layers.Dropout(0.2)(branch7x7dbl)
  branch7x7dbl = conv2d_bn(branch7x7dbl, 128, (1,7), 1)
  # branch7x7dbl=layers.Dropout(0.2)(branch7x7dbl)
  branch7x7dbl = conv2d_bn(branch7x7dbl, 128, (7,1), 1)
  # branch7x7dbl=layers.Dropout(0.2)(branch7x7dbl)
  branch7x7dbl = conv2d_bn(branch7x7dbl, 128, (1,7), 1)

  branch_pool = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
  branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
  x = layers.concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool],axis=channel_axis,name='mixed4')

  # x=layers.Dropout(0.4)(x)


  x=layers.Dropout(0.2)(x)
  # mixed 5,6  batch_size,17 x 17 x 768
  for i in range(2):
    branch1x1 = conv2d_bn(x, 192, 1, 1)

    branch7x7 = conv2d_bn(x, 160, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 128, (1,7), 1)
    branch7x7 = conv2d_bn(branch7x7, 128, (7,1), 1)

    branch7x7dbl = conv2d_bn(x, 140, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, (7,1), 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, (1,7), 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, (7,1), 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, (1,7), 1)

    branch_pool =layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)

    branch_pool = conv2d_bn(branch_pool, 128, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch7x7, branch7x7dbl, branch_pool],
        axis=channel_axis,
        name='mixed' + str(5 + i))
    if i==0:
      x=layers.Dropout(0.4)(x)


  #  # mixed 7: 17 x 17 x 768
  # branch1x1 = conv2d_bn(x, 192, 1, 1)

  # branch7x7 = conv2d_bn(x, 192, 1, 1)
  # branch7x7 = conv2d_bn(branch7x7, 192, (1,7), 1)
  # branch7x7 = conv2d_bn(branch7x7, 192, (7,1), 1)

  # branch7x7dbl = conv2d_bn(x, 192, 1, 1)
  # branch7x7dbl = conv2d_bn(branch7x7dbl, 192, (1,7), 1)
  # branch7x7dbl = conv2d_bn(branch7x7dbl, 192, (7,1), 1)
  # branch7x7dbl = conv2d_bn(branch7x7dbl, 192, (1,7), 1)
  # branch7x7dbl = conv2d_bn(branch7x7dbl, 192, (7,1), 1)

  # branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
  # branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
  # x = layers.concatenate(
  #     [branch1x1, branch7x7, branch7x7dbl, branch_pool],
  #     axis=channel_axis,
  #     name='mixed7')
  
  # x=layers.Dropout(0.2)(x)
  
  x=conv2d_bn(x, 192, 1, 1,padding='same')

  x=conv2d_bn(x, 64, 3, 1,padding='valid')
  x=layers.Dropout(0.2)(x)
 
  # x=conv2d_bn(x, 192, 3, 1,padding='same')
  # # x=layers.Dropout(0.2)(x)

  # x=conv2d_bn(x, 64, 1, 1,padding='valid')
  # # x=layers.Dropout(0.2)(x)

  x=conv2d_bn(x, 64, 5, 2,padding='same')
  # x=layers.Dropout(0.2)(x)

  x=conv2d_bn(x, 32, 1, 1,padding='valid')
  x=conv2d_bn(x, 64, 5, 2,padding='same')
  # x=layers.Dropout(0.2)(x)

  x=conv2d_bn(x, 128, 3, 2,padding='valid')
  # x=conv2d_bn(x, 32, 1, 1,padding='valid')
  # x=conv2d_bn(x, 64, 2, 1,padding='valid')
  # x=layers.Dropout(0.3)(x)
  
  x=layers.Flatten()(x)

  # x=tfa.layers.SpectralNormalization(layers.Dense(units=1024,use_bias=False,kernel_regularizer=tf.keras.regularizers.L2(0.1),kernel_initializer='random_normal'))(x)
  # x=layers.Activation('LeakyReLU')(x)
  x=layers.Dropout(0.5)(x)

  x=layers.Dense(units=1)(x)

  return keras.Model(inputs=[label_inp_layer,inp], outputs=x)

discriminator=discriminator()




discriminator.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 16)        80          ['input_2[0][0]']                
                                                                                                  
 spectral_normalization (Spectr  (None, 1, 16)       288         ['embedding[0][0]']              
 alNormalization)                                                                                 
                                                                                                  
 flatten (Flatten)              (None, 16)           0           ['spectral_normalization[0][0

In [ ]:
def generator():

  # kernel_regularizer=regularizers.L2(1e-4)
  
  initializer =tf.keras.initializers.RandomNormal()

  label_inp_layer=layers.Input(shape=(1,))
  x=layers.Embedding(num_classes,16)(label_inp_layer)
  # x=Dense(1*1*256)(x)
  # x=layers.LeakyReLU(alpha=0.2)(x)
  x=layers.Reshape((1,1,16))(x)

  inp =layers.Input(shape=(noise_size,))
  # y=layers.Dense(1 * 1 * latent_dim)(inp)
  # y=layers.LeakyReLU(alpha=0.2)(y)
  y=layers.Reshape((1, 1, noise_size))(inp)
  
  y=layers.concatenate([y, x],axis=-1)

  y=layers.Dense(256)(y)
  y=layers.LeakyReLU(alpha=0.2)(y)
 

  #(batch_size, 2, 2,514)
  y= layers.Conv2DTranspose(200,4,strides=2,padding='same',use_bias=False)(y)
  y=layers.BatchNormalization(scale=False)(y)
  y=layers.LeakyReLU()(y)
  y=layers.Dropout(.5)(y)

  #(batch_size, 4, 4,514)
  y= layers.Conv2DTranspose(200,4,strides=2,padding='same',use_bias=False)(y)
  y=layers.BatchNormalization(scale=False)(y)
  y=layers.LeakyReLU()(y)
  y=layers.Dropout(.3)(y)

  #(batch_size, 8, 8,514)
  y= layers.Conv2DTranspose(200,4,strides=2,padding='same',use_bias=False)(y)
  y=layers.BatchNormalization(scale=False)(y)
  y=layers.LeakyReLU()(y)
  y=layers.Dropout(.5)(y)

  #(batch_size, 16, 16,514) 50=128
  y= layers.Conv2DTranspose(200,4,strides=2,padding='same',use_bias=False)(y)
  y=layers.BatchNormalization(scale=False)(y)
  y=layers.LeakyReLU()(y)
  y=layers.Dropout(.3)(y)


  #(batch_size, 32, 32,514)
  y= layers.Conv2DTranspose(200,4,strides=2,padding='same',use_bias=False )(y)#tf.random_normal_initializer(0., 0.05)
  y=layers.BatchNormalization(scale=False)(y)
  y=layers.LeakyReLU()(y)
  y=layers.Dropout(.3)(y)


  #(batch_size, 64, 64,514)
  y= layers.Conv2DTranspose(200,4,strides=2,padding='same',use_bias=False)(y)#,kernel_regularizer=regularizers.L2(1e-4)
  y=layers.BatchNormalization(scale=False)(y)
  y=layers.LeakyReLU()(y)
  y=layers.Dropout(.3)(y)

  #(batch_size, 128, 128,514)
  y= layers.Conv2DTranspose(200,4,strides=2,padding='same',use_bias=False)(y)
  y=layers.BatchNormalization(scale=False)(y)
  y=layers.LeakyReLU()(y)
  y=layers.Dropout(.3)(y)
  
  #(batch_size, 256, 256,3)
  y=layers.Conv2DTranspose(3,4,strides=2,padding='same',activation='tanh')(y)

  return keras.Model(inputs=[label_inp_layer,inp], outputs=y)

generator=generator()

# tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)
generator.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 1, 16)        80          ['input_3[0][0]']                
                                                                                                  
 reshape_2 (Reshape)            (None, 1, 1, 256)    0           ['input_4[0][0]']                
                                                                                            

In [ ]:
kid_image_size=75
ema = 0.99

class KID(keras.metrics.Metric):
    def __init__(self, name="kid", **kwargs):
        super().__init__(name=name, **kwargs)

        # KID is estimated per batch and is averaged across batches
        self.kid_tracker = keras.metrics.Mean()

        # a pretrained InceptionV3 is used without its classification layer
        # transform the pixel values to the 0-255 range, then use the same
        # preprocessing as during pretraining
        self.encoder = keras.Sequential(
            [
                layers.InputLayer(input_shape=(image_size, image_size, 3)),
                layers.Rescaling(255.0),
                layers.Resizing(height=kid_image_size, width=kid_image_size),
                layers.Lambda(keras.applications.inception_v3.preprocess_input),
                keras.applications.InceptionV3(
                    include_top=False,
                    input_shape=(kid_image_size, kid_image_size, 3),
                    weights="imagenet",
                ),
                layers.GlobalAveragePooling2D(),
            ],
            name="inception_encoder",
        )

    def polynomial_kernel(self, features_1, features_2):
        feature_dimensions = tf.cast(tf.shape(features_1)[1], dtype=tf.float32)
        return (features_1 @ tf.transpose(features_2) / feature_dimensions + 1.0) ** 3.0

    def update_state(self, real_images, generated_images, sample_weight=None):
        real_features = self.encoder(real_images, training=False)
        generated_features = self.encoder(generated_images, training=False)

        # compute polynomial kernels using the two sets of features
        kernel_real = self.polynomial_kernel(real_features, real_features)
        kernel_generated = self.polynomial_kernel(
            generated_features, generated_features
        )
        kernel_cross = self.polynomial_kernel(real_features, generated_features)

        # estimate the squared maximum mean discrepancy using the average kernel values
        batch_size = tf.shape(real_features)[0]
        batch_size_f = tf.cast(batch_size, dtype=tf.float32)
        mean_kernel_real = tf.reduce_sum(kernel_real * (1.0 - tf.eye(batch_size))) / (
            batch_size_f * (batch_size_f - 1.0)
        )
        mean_kernel_generated = tf.reduce_sum(
            kernel_generated * (1.0 - tf.eye(batch_size))
        ) / (batch_size_f * (batch_size_f - 1.0))
        mean_kernel_cross = tf.reduce_mean(kernel_cross)
        kid = mean_kernel_real + mean_kernel_generated - 2.0 * mean_kernel_cross

        # update the average KID estimate
        self.kid_tracker.update_state(kid)

    def result(self):
        return self.kid_tracker.result()

    def reset_state(self):
        self.kid_tracker.reset_state()


In [ ]:
max_translation = 0.125
max_rotation = 0.125
max_zoom = 0.15
target_accuracy = 0.85
integration_steps = 1000
#  "hard sigmoid", useful for binary accuracy calculation from logits

def step(values):
    # negative values -> 0.0, positive values -> 1.0
    return 0.5 * (1.0 + tf.sign(values))


# augments images with a probability that is dynamically updated during training

class AdaptiveAugmenter(keras.Model):
    def __init__(self):
        super().__init__()

        # stores the current probability of an image being augmented
        self.probability = tf.Variable(0.0)

        # the corresponding augmentation names from the paper are shown above each layer
        # the authors show (see figure 4), that the blitting and geometric augmentations
        # are the most helpful in the low-data regime
        self.augmenter = keras.Sequential(
            [
                layers.InputLayer(input_shape=(image_size, image_size, 3)),
                # blitting/x-flip:
                layers.RandomFlip("horizontal"),
                # blitting/integer translation:
                layers.RandomTranslation(
                    height_factor=max_translation,
                    width_factor=max_translation,
                    interpolation="nearest",
                ),
                # geometric/rotation:
                layers.RandomRotation(factor=max_rotation),
                # geometric/isotropic and anisotropic scaling:
                layers.RandomZoom(
                    height_factor=(-max_zoom, 0.0), width_factor=(-max_zoom, 0.0)
                ),
            ],
            name="adaptive_augmenter",
        )

    def call(self, images, training):
        if training:
            augmented_images = self.augmenter(images, training)

            # during training either the original or the augmented images are selected
            # based on self.probability
            augmentation_values = tf.random.uniform(
                shape=(batch_size, 1, 1, 1), minval=0.0, maxval=1.0
            )
            augmentation_bools = tf.math.less(augmentation_values, self.probability)

            images = tf.where(augmentation_bools, augmented_images, images)
        return images

    def update(self, real_logits):
        current_accuracy = tf.reduce_mean(step(real_logits))

        # the augmentation probability is updated based on the dicriminator's
        # accuracy on real images
        accuracy_error = current_accuracy - target_accuracy
        self.probability.assign(
            tf.clip_by_value(
                self.probability + accuracy_error / integration_steps, 0.0, 1.0
            )
        )

In [ ]:
class ConditionalGAN(tf.keras.Model):
  def __init__(self,discriminator, generator):
    super(ConditionalGAN, self).__init__()
    self.discriminator = discriminator
    self.generator = generator
    self.ema_generator = keras.models.clone_model(self.generator)
    self.augmenter = AdaptiveAugmenter()
    
    
    
  def compile(self, d_optimizer, g_optimizer):
    super(ConditionalGAN,self).compile()
    self.d_optimizer = d_optimizer
    self.g_optimizer = g_optimizer

    # self.generator_loss = generator_loss
    # self.critic_loss=critic_loss
    self.generator_loss_tracker = keras.metrics.Mean(name="g_loss")
    self.discriminator_loss_tracker = keras.metrics.Mean(name="d_loss")
    self.real_accuracy = keras.metrics.BinaryAccuracy(name="real_acc")
    self.generated_accuracy = keras.metrics.BinaryAccuracy(name="gen_acc")
    self.augmentation_probability_tracker = keras.metrics.Mean(name="aug_p")

    self.kid = KID()
   
  @property
  def metrics(self):
    return [self.generator_loss_tracker,
            self.discriminator_loss_tracker,
            self.real_accuracy,
            self.generated_accuracy,
            self.augmentation_probability_tracker,
            self.kid,]   

  @tf.function
  def adversarial_loss(self, real_logits, generated_logits):
    # this is usually called the non-saturating GAN loss

    real_labels = tf.ones(shape=(batch_size, 1))
    generated_labels = tf.zeros(shape=(batch_size, 1))

    # the generator tries to produce images that the discriminator considers as real
    generator_loss = keras.losses.binary_crossentropy(
        real_labels, generated_logits, from_logits=True
    )
    # the discriminator tries to determine if images are real or generated
    discriminator_loss = keras.losses.binary_crossentropy(
        tf.concat([real_labels, generated_labels], axis=0),
        tf.concat([real_logits, generated_logits], axis=0),
        from_logits=True,
    )

    return tf.reduce_mean(generator_loss), tf.reduce_mean(discriminator_loss)
    
  @tf.function
  def generate(self,labels, training, batch_size=batch_size):
    latent_samples = tf.random.normal(shape=(batch_size, noise_size))
    # use ema_generator during inference
    if training:
        generated_images = self.generator([labels,latent_samples], training)
    else:
        generated_images = self.ema_generator([labels,latent_samples], training)
    return generated_images

  @tf.function
  def train_step(self, data):
    real_images,labels=data
    real_images = self.augmenter(real_images, training=True)

    # use persistent gradient tape because gradients will be calculated twice
    with tf.GradientTape(persistent=True) as tape:
      generated_images = self.generate(labels,batch_size=batch_size,training=True)
      # gradient is calculated through the image augmentation
      generated_images = self.augmenter(generated_images, training=True)

      # separate forward passes for the real and generated images, meaning
      # that batch normalization is applied separately
      real_logits = self.discriminator([labels,real_images], training=True)
      generated_logits = self.discriminator([labels,generated_images], training=True)

      generator_loss, discriminator_loss = self.adversarial_loss(
          real_logits, generated_logits
      )

    # calculate gradients and update weights
    generator_gradients = tape.gradient(
        generator_loss, self.generator.trainable_weights
    )
    discriminator_gradients = tape.gradient(
        discriminator_loss, self.discriminator.trainable_weights
    )
    self.g_optimizer.apply_gradients(
        zip(generator_gradients, self.generator.trainable_weights)
    )
    self.d_optimizer.apply_gradients(
        zip(discriminator_gradients, self.discriminator.trainable_weights)
    )

    # update the augmentation probability based on the discriminator's performance
    self.augmenter.update(real_logits)

    self.generator_loss_tracker.update_state(generator_loss)
    self.discriminator_loss_tracker.update_state(discriminator_loss)
    self.real_accuracy.update_state(1.0, step(real_logits))
    self.generated_accuracy.update_state(0.0, step(generated_logits))
    self.augmentation_probability_tracker.update_state(self.augmenter.probability)

    # track the exponential moving average of the generator's weights to decrease
    # variance in the generation quality
    for weight, ema_weight in zip(self.generator.weights, self.ema_generator.weights):
      ema_weight.assign(ema * ema_weight + (1 - ema) * weight)

    # KID is not measured during the training phase for computational efficiency
    return {m.name: m.result() for m in self.metrics[:-1]}

    
  def test_step(self, data):
    real_images,labels=data
    generated_images = self.generate(labels,batch_size=batch_size, training=False)

    self.kid.update_state(real_images, generated_images)

    # only KID is measured during the evaluation phase for computational efficiency
    return {self.kid.name: self.kid.result()}

  def plot_images(self, epoch=None, logs=None, num_rows=1, num_cols=5, interval=5):
    # plot random generated images for visual evaluation of generation quality
    if epoch is None or (epoch + 1) % interval == 0:
      num_images = num_rows * num_cols
      generated_images = self.generate(np.array([0,1,2,3,4]),batch_size=5, training=False)

      plt.figure(figsize=(num_cols * 2.0, num_rows * 2.0))
      for row in range(num_rows):
        for col in range(num_cols):
          index = row * num_cols + col
          plt.subplot(num_rows, num_cols, index + 1)
          plt.imshow(generated_images[index])
          plt.axis("off")
      plt.tight_layout()
      plt.show()
      plt.close()


In [ ]:
checkpoint_dir = '/content/drive/MyDrive/GAN GENERATED/chk'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(generator=generator,discriminator=discriminator)

manager = tf.train.CheckpointManager(
    checkpoint, directory=checkpoint_dir, max_to_keep=3)

In [ ]:
# #restoring checkpoint. verify using assert below

# checkpoint_dir=('/content/drive/MyDrive/GAN GENERATED/chk')

# #checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir)).assert_consumed()

# checkpoint.restore(manager.latest_checkpoint).assert_consumed()

In [ ]:
import time
%matplotlib inline
class GANMonitor(keras.callbacks.Callback):

  # def on_epoch_begin(self, epoch, logs=None):
  #   global epochh
  #   epochh=epoch+1
    
  def on_epoch_end(self, epoch, logs=None):
  #   print('GAN MONITOR',logs.keys)
    if (epoch+1) % 15 == 0:
      manager.save()

    if (epoch+1) % 5 == 0:
      pth=['laptop', 'keyboard', 'mouse', 'headphone', 'monitor']
      path=['/content/drive/MyDrive/GAN GENERATED/'+i for i in pth]
            
      classes=[[0],[1],[2],[3],[4]]
      classes=np.array(classes,dtype=np.int8)  #(5, 1)

      generated_images = self.model.generate(classes,batch_size=5, training=False)
      generated_images = (generated_images*127.5)+127.5

      for i in range(num_classes):
        img = generated_images[i].numpy()
        img = keras.preprocessing.image.array_to_img(img) 
      
        img.save(path[i]+'/'+str(epoch+1)+'.png')
      
    
    

monitor=GANMonitor()

In [ ]:
model = ConditionalGAN( discriminator=discriminator , generator=generator)
Lr=0.0002
model.compile(
    g_optimizer= keras.optimizers.Adam(learning_rate=Lr, beta_1=0.5, beta_2=0.9),
    d_optimizer= keras.optimizers.Adam(learning_rate=Lr, beta_1=0.5, beta_2=0.9),
)

# save the best model based on the validation KID metric
checkpoint_dir = '/content/drive/MyDrive/GAN GENERATED'
checkpoint_path = os.path.join(checkpoint_dir, "ckpt")

csv_logger=tf.keras.callbacks.CSVLogger('/content/drive/MyDrive/GAN GENERATED/log.csv', separator=",", append=True)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor="val_kid",
    mode="min",
    save_best_only=True,
)

# run training and plot generated images periodically
model.fit(
    dataset,
    epochs=1000,
    steps_per_epoch=38,
    validation_steps=5,
    validation_data=dataset,
    callbacks=[monitor,csv_logger,
        # keras.callbacks.LambdaCallback(on_epoch_end=model.plot_images),
        checkpoint_callback,
    ],
)

Epoch 1/1000
38/38 [==============================] - 468s 11s/step - g_loss: 0.6706 - d_loss: 1.1307 - real_acc: 0.5164 - gen_acc: 0.4762 - aug_p: 3.3865e-04 - val_kid: 6.6368
Epoch 2/1000
38/38 [==============================] - 49s 1s/step - g_loss: 0.7014 - d_loss: 0.6931 - real_acc: 0.0000e+00 - gen_acc: 1.0000 - aug_p: 0.0000e+00 - val_kid: 3.7126
Epoch 3/1000
38/38 [==============================] - 48s 1s/step - g_loss: 0.6647 - d_loss: 0.6811 - real_acc: 0.5395 - gen_acc: 0.5831 - aug_p: 2.2023e-04 - val_kid: 5.2697
Epoch 4/1000
38/38 [==============================] - 48s 1s/step - g_loss: 0.7579 - d_loss: 0.5519 - real_acc: 0.5189 - gen_acc: 0.7944 - aug_p: 1.7253e-04 - val_kid: 3.4059
Epoch 5/1000
38/38 [==============================] - 56s 1s/step - g_loss: 0.9584 - d_loss: 0.7870 - real_acc: 0.2549 - gen_acc: 0.8734 - aug_p: 3.4868e-05 - val_kid: 3.9116
Epoch 6/1000
38/38 [==============================] - 48s 1s/step - g_loss: 0.7840 - d_loss: 0.6640 - real_acc: 0.2673 